# Predicting Asset Option Price (American Binomial Options Method)

### Project Overview
In this case ABC company uses binomial American call option pricing to determine the fair value of Company XYZ's participation rights in the X project. 

#### What is Options?

The term option refers to a financial instrument that is based on the value of underlying securities such as stocks, indexes, and exchange traded funds (ETFs). An options contract offers the buyer the opportunity to buy or sell—depending on the type of contract they hold—the underlying asset. Unlike futures, the holder is not required to buy or sell the asset if they decide against it. Each options contract will have a specific expiration date by which the holder must exercise their option. The stated price on an option is known as the strike price.

#### What is American Options?
An American option, aka an American-style option, is a version of an options contract that allows holders to exercise the option rights at any time before and including the day of expiration. It contrasts with another type of option, called the European option, that only allows execution on the day of expiration.

#### The formula for calculation the option price is based on several parameters:

* Budgeted Capex Cost = Base Price * (1 + Interest) * FX Adjustment * Incentives * Nameplate Capacity
* Excersie Price = (Budgeted Capex Cost - Capex Loan - Accrued Capex Cost) - First PR Deduction
* Option Price = max[Underlying Asset Value - Excersie Price, 0]
* Valuation Date = Mechanical Completion Date
* Cut Off Date = 14 Feb 2024 (assumption)

### Business Understanding

We note that ABC assumes that the asset price of X can either up or down and has a terminal value. However, based on information from XYZ management, the X project has a duration of only **20 years**, making it unlikely for the project to have a terminal value. Additionally, the stock price of the X project is likely to decline over time, as future cash flows obtained by XYZ decrease.

### Libraries

In [1]:
import pandas as pd
import numpy as np

### Inputs and Assumptions

Due to the XYZ management, the participating right is set to 5 years. During this duration ABC has the option to buy the underlying asset.

In [2]:
period = 5 # participating right duration in years
numberofDays = 252 * period #252 trading days each year (assumptions)

The parameters to determine the budgeted capex cost of Project X are adjusted every 6 months during the participation right. Therefore, we need to create a 6-month array before assigning the parameters into the daily array.

In [3]:
# 6 month parameters based on definitive agreement
basePrice_6month          = [29000,    29000,    29000,    29000,     29000,     29000,      29000,      29000,      29000,      29000,      29000  ]
interest_6month           = [0.00,     0.04,     0.08,     0.12,      0.16,      0.20,       0.24,       0.28,       0.32,       0.36,       0.40   ]
fixAdjustment_6month      = [1.0203,   1.0203,   1.0217,   1.0217,    1.0217,    1.0217,     1.0217,     1.0217,     1.0217,     1.0217,     1.0217 ]
incentives_6month         = [0,        0,        0.05,     0.05,      0.10,      0.10,       0.10,       0.10,       0.05,       0.05,       0.05   ]
nameplateCapacity_6month  = [120000,   120000,   120000,   120000,    120000,    120000,     120000,     120000,     120000,     120000,     120000 ]
capexLoan_6month          = [0,        2096,     1796,     1497,      1197,      898,        599,        299,        0,          0,          0      ]
accruedCapex_6month       = [10,       10,       10,       10,        10,        10,         10,         10,         10,         10,         10     ]
firstprDeduction_6month   = [12.26,    12.26,    12.26,    12.26,     12.26,     12.26,      12.26,      12.26,      12.26,      12.26,      12.26  ]

From this 6-month information, we generate the daily array

In [4]:
# initialize daily parameters
basePrice = np.zeros(numberofDays+1)
interest = np.zeros(numberofDays+1)
fxAdjustment = np.zeros(numberofDays+1)
incentives = np.zeros(numberofDays+1)
nameplateCapacity = np.zeros(numberofDays+1)
capexLoan = np.zeros(numberofDays+1)
accruedCapex = np.zeros(numberofDays+1)
firstprDeduction = np.zeros(numberofDays+1)

In [5]:
def sixmonthToDaily(dailyArray,sixmonthArray):
  # convert from 6-month period to daily period
  j = 0
  for i in range(len(dailyArray)):
    if i>0 and (i % (6 * 21) == 0):
      j = j + 1
    dailyArray[i] = sixmonthArray[j]
  return dailyArray

In [6]:
# caculate full daily parameters
basePrice = sixmonthToDaily(basePrice,basePrice_6month)
interest = sixmonthToDaily(interest,interest_6month)
fxAdjustment = sixmonthToDaily(fxAdjustment,fixAdjustment_6month)
incentives = sixmonthToDaily(incentives,incentives_6month)
nameplateCapacity = sixmonthToDaily(nameplateCapacity,nameplateCapacity_6month)
capexLoan = sixmonthToDaily(capexLoan,capexLoan_6month)
accruedCapex = sixmonthToDaily(accruedCapex,accruedCapex_6month)
firstprDeduction = sixmonthToDaily(firstprDeduction,firstprDeduction_6month)

The budgeted capex cost and exercise price are determined based on the formula defined in the agreement.

In [7]:
# calculate budgeted capex
budgetedCapex = basePrice * ((np.ones(numberofDays+1) + interest) * fxAdjustment + incentives) * nameplateCapacity # based on definitive agreement
budgetedCapex = budgetedCapex / 1e6 # convert to million usd   

print(f"Budgeted Capex in Million USD: {budgetedCapex}")

# calculate exercise price
exercisePrice = budgetedCapex - capexLoan - accruedCapex - firstprDeduction
exercisePrice = 0.3 * exercisePrice # 30% of underlying asset will be traded based on definitive agreement

print(f"Exercise Price in Million USD: {exercisePrice}")

Budgeted Capex in Million USD: [3550.644   3550.644   3550.644   ... 5009.50176 5009.50176 5151.7224 ]
Exercise Price in Million USD: [1058.5152   1058.5152   1058.5152   ... 1496.172528 1496.172528
 1538.83872 ]


### Calculate Option Price

In [8]:
# option price parameters
volatility = 0.5282
rf = 0.0438 # risk free rate
dt = period / numberofDays # delta time
underlyingAsset0 = 3683.1 * 0.3 # 30% of underlying asset will be traded based on definitive agreement

up = np.exp(volatility * np.sqrt(dt))
down = 1 / up
pu = (np.exp(rf * dt) - down) / (up - down) # probability up
pd = 1 - pu # probability down

In [9]:
# initialize option price and underlying asset price
optionPrice = np.zeros((numberofDays+1, numberofDays+1))
underlyingAsset = np.zeros((numberofDays+1, numberofDays+1))

In [10]:
# calculate option price and underlying asset price movement
for j in range(numberofDays + 1):
  underlyingAsset[j, numberofDays] = underlyingAsset0 * (up ** (numberofDays - j)) * (down ** j)

for i in range(numberofDays - 1, -1, -1):
  for j in range(i + 1):
    underlyingAsset[j, i] = underlyingAsset[j, i + 1] / up
    
for j in range(numberofDays + 1):
  optionPrice[j, numberofDays] = max(0, underlyingAsset[j, numberofDays] - exercisePrice[j])

for i in range(numberofDays - 1, -1, -1):
  for j in range(i + 1):
    optionPrice[j, i] = max(0, underlyingAsset[j, i] - exercisePrice[j], np.exp(-rf * dt) * (pu * optionPrice[j, i + 1] + (1 - pu) * optionPrice[j + 1, i + 1]))


print(f"Option Price: {optionPrice[0][0]} million USD")

Option Price: 710.0146079070705 million USD


Based on the binomial american option calculation, the option price of the underlying asset during participating right is **USD 710 Million**